# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
# !pip3 install xlsxwriter  

In [5]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 137236063,
 'avg30Volume': 122516518,
 'beta': 1.3922606634262864,
 'companyName': 'Apple Inc',
 'day200MovingAvg': 151.88,
 'day30ChangePercent': 0.1449439506473411,
 'day50MovingAvg': 162.09,
 'day5ChangePercent': -0.013559799303388345,
 'dividendYield': 0.0050785559860862,
 'employees': 152974,
 'exDividendDate': '2021-10-28',
 'float': 0,
 'marketcap': 2870351843748,
 'maxChangePercent': 66.78341445005746,
 'month1ChangePercent': 0.1436434530236435,
 'month3ChangePercent': 0.16079908531191173,
 'month6ChangePercent': 0.33845556641270436,
 'nextDividendDate': '',
 'nextEarningsDate': '2022-01-26',
 'peRatio': 15.22639320135563,
 'sharesOutstanding': 16991225784,
 'ttmDividendRate': 0.8762243493906864,
 'ttmEPS': 11.25,
 'week52change': 0.3582841136269463,
 'week52high': 189.48,
 'week52highSplitAdjustOnly': 182.92,
 'week52low': 120.47,
 'week52lowSplitAdjustOnly': 118.88,
 'year1ChangePercent': 0.3679483331101672,
 'year2ChangePercent': 1.529504508176411,
 'year5Cha

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [8]:
data['year1ChangePercent']

0.3679483331101672

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    # print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_call_url)
    if(response.status_code==200):
      data = response.json()
      for symbol in symbol_string.split(','):
        if(symbol!='KSU'):
          final_dataframe = final_dataframe.append(
                                          pd.Series([symbol, 
                                                    data[symbol]['quote']['latestPrice'],
                                                    data[symbol]['stats']['year1ChangePercent'],
                                                    'N/A'
                                                    ], 
                                                    index = my_columns), 
                                          ignore_index = True)
          
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,153.52,0.300159,N/A
1,AAL,17.63,-0.0205944,N/A
2,AAP,237.61,0.526274,N/A
3,AAPL,175.80,0.36972,N/A
4,ABBV,135.94,0.326858,N/A
...,...,...,...,...
499,YUM,137.54,0.268128,N/A
500,ZBH,125.12,-0.185924,N/A
501,ZBRA,607.24,0.592978,N/A
502,ZION,64.40,0.557042,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [11]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,83.070,2.36051,N/A
1,DVN,41.180,1.73086,N/A
2,MCHP,88.430,1.47383,N/A
3,FTNT,337.020,1.47281,N/A
4,FANG,104.000,1.29952,N/A
5,F,20.690,1.27285,N/A
6,MRO,15.240,1.23746,N/A
7,NVDA,293.910,1.14718,N/A
8,IT,332.150,1.11407,N/A
9,NUE,117.910,1.11387,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Price'])
final_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,83.070,2.36051,2407
1,DVN,41.180,1.73086,4856
2,MCHP,88.430,1.47383,2261
3,FTNT,337.020,1.47281,593
4,FANG,104.000,1.29952,1923
5,F,20.690,1.27285,9666
6,MRO,15.240,1.23746,13123
7,NVDA,293.910,1.14718,680
8,IT,332.150,1.11407,602
9,NUE,117.910,1.11387,1696


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [14]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [15]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
      if(symbol!='KSU'):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.270,N/A,0.307036,N/A,0.0546351,N/A,-0.131446,N/A,-0.0657662,N/A,N/A
1,AAL,17.610,N/A,-0.0207342,N/A,-0.289587,N/A,-0.170551,N/A,-0.16772,N/A,N/A
2,AAP,242.960,N/A,0.521494,N/A,0.233837,N/A,0.161659,N/A,0.0191779,N/A,N/A
3,AAPL,178.244,N/A,0.362319,N/A,0.327599,N/A,0.16271,N/A,0.144981,N/A,N/A
4,ABBV,134.090,N/A,0.327845,N/A,0.173697,N/A,0.241681,N/A,0.133446,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,134.050,N/A,0.267144,N/A,0.145723,N/A,0.0487133,N/A,0.0574425,N/A,N/A
500,ZBH,125.780,N/A,-0.192798,N/A,-0.235416,N/A,-0.180141,N/A,-0.0877577,N/A,N/A
501,ZBRA,605.900,N/A,0.569278,N/A,0.174685,N/A,0.0125455,N/A,-0.0224469,N/A,N/A
502,ZION,62.800,N/A,0.571928,N/A,0.16906,N/A,0.110677,N/A,-0.0468108,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [16]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for i in hqm_dataframe.index:
    for time_period in time_periods:
      change_col = f'{time_period} Price Return'
      if hqm_dataframe.loc[i, change_col] == None: # to prevent comparision b/w NoneType and float error
        hqm_dataframe.loc[i, change_col] = 0 

for i in hqm_dataframe.index:
    for time_period in time_periods:
      change_col = f'{time_period} Price Return'
      percentile_col = f'{time_period} Return Percentile'
      hqm_dataframe.loc[i, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[i, change_col])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.571429
1       0.123016
2       0.819444
3       0.656746
4       0.611111
         ...    
499     0.509921
500    0.0357143
501     0.859127
502     0.865079
503      0.77381
Name: One-Year Return Percentile, Length: 504, dtype: object
0       0.480159
1       0.031746
2       0.861111
3       0.934524
4       0.748016
         ...    
499     0.696429
500    0.0436508
501         0.75
502     0.740079
503     0.904762
Name: Six-Month Return Percentile, Length: 504, dtype: object
0      0.0813492
1      0.0515873
2       0.869048
3       0.871032
4       0.954365
         ...    
499      0.53373
500    0.0396825
501     0.414683
502     0.753968
503     0.859127
Name: Three-Month Return Percentile, Length: 504, dtype: object
0       0.230159
1      0.0396825
2       0.674603
3       0.992063
4       0.984127
         ...    
499     0.843254
500     0.152778
501     0.440476
502     0.297619
503      0.84127
Name: One-Month Return Percentile, Length: 504, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.270,N/A,0.307036,0.571429,0.0546351,0.480159,-0.131446,0.0813492,-0.0657662,0.230159,N/A
1,AAL,17.610,N/A,-0.0207342,0.123016,-0.289587,0.031746,-0.170551,0.0515873,-0.16772,0.0396825,N/A
2,AAP,242.960,N/A,0.521494,0.819444,0.233837,0.861111,0.161659,0.869048,0.0191779,0.674603,N/A
3,AAPL,178.244,N/A,0.362319,0.656746,0.327599,0.934524,0.16271,0.871032,0.144981,0.992063,N/A
4,ABBV,134.090,N/A,0.327845,0.611111,0.173697,0.748016,0.241681,0.954365,0.133446,0.984127,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,134.050,N/A,0.267144,0.509921,0.145723,0.696429,0.0487133,0.53373,0.0574425,0.843254,N/A
500,ZBH,125.780,N/A,-0.192798,0.0357143,-0.235416,0.0436508,-0.180141,0.0396825,-0.0877577,0.152778,N/A
501,ZBRA,605.900,N/A,0.569278,0.859127,0.174685,0.75,0.0125455,0.414683,-0.0224469,0.440476,N/A
502,ZION,62.800,N/A,0.571928,0.865079,0.16906,0.740079,0.110677,0.753968,-0.0468108,0.297619,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [17]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,156.270,N/A,0.307036,0.571429,0.0546351,0.480159,-0.131446,0.0813492,-0.0657662,0.230159,0.340774
1,AAL,17.610,N/A,-0.0207342,0.123016,-0.289587,0.031746,-0.170551,0.0515873,-0.16772,0.0396825,0.0615079
2,AAP,242.960,N/A,0.521494,0.819444,0.233837,0.861111,0.161659,0.869048,0.0191779,0.674603,0.806052
3,AAPL,178.244,N/A,0.362319,0.656746,0.327599,0.934524,0.16271,0.871032,0.144981,0.992063,0.863591
4,ABBV,134.090,N/A,0.327845,0.611111,0.173697,0.748016,0.241681,0.954365,0.133446,0.984127,0.824405
...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,134.050,N/A,0.267144,0.509921,0.145723,0.696429,0.0487133,0.53373,0.0574425,0.843254,0.645833
500,ZBH,125.780,N/A,-0.192798,0.0357143,-0.235416,0.0436508,-0.180141,0.0396825,-0.0877577,0.152778,0.0679563
501,ZBRA,605.900,N/A,0.569278,0.859127,0.174685,0.75,0.0125455,0.414683,-0.0224469,0.440476,0.616071
502,ZION,62.800,N/A,0.571928,0.865079,0.16906,0.740079,0.110677,0.753968,-0.0468108,0.297619,0.664187


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [24]:
hqm_dataframe = hqm_dataframe.sort_values('HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ACN,416.760,479,0.644864,0.918651,0.422398,0.980159,0.178872,0.888889,0.0820522,0.916667,0.926091
1,AAPL,178.244,1122,0.362319,0.656746,0.327599,0.934524,0.16271,0.871032,0.144981,0.992063,0.863591
2,ANET,140.470,1423,0.830862,0.970238,0.415192,0.97619,0.445154,0.994048,-0.0131185,0.492063,0.858135
3,AOS,82.090,2436,0.554805,0.855159,0.240491,0.867063,0.275209,0.970238,0.0157225,0.652778,0.83631
4,ABBV,134.090,1491,0.327845,0.611111,0.173697,0.748016,0.241681,0.954365,0.133446,0.984127,0.824405
5,ANTM,453.350,441,0.465755,0.771825,0.194193,0.78373,0.206477,0.926587,0.0448434,0.779762,0.815476
6,AAP,242.960,823,0.521494,0.819444,0.233837,0.861111,0.161659,0.869048,0.0191779,0.674603,0.806052
7,AVB,251.658,794,0.596559,0.882937,0.193254,0.779762,0.111459,0.757937,0.017225,0.660714,0.770337
8,AMP,308.660,647,0.638442,0.914683,0.199289,0.787698,0.111474,0.759921,-0.0280286,0.40873,0.717758
9,ARE,223.100,896,0.256267,0.503968,0.210492,0.829365,0.103856,0.72619,0.0515945,0.809524,0.717262


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [25]:
portfolio_input()

Enter the value of your portfolio:1000000


In [26]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ACN,416.760,47,0.644864,0.918651,0.422398,0.980159,0.178872,0.888889,0.0820522,0.916667,0.926091
1,AAPL,178.244,112,0.362319,0.656746,0.327599,0.934524,0.16271,0.871032,0.144981,0.992063,0.863591
2,ANET,140.470,142,0.830862,0.970238,0.415192,0.97619,0.445154,0.994048,-0.0131185,0.492063,0.858135
3,AOS,82.090,243,0.554805,0.855159,0.240491,0.867063,0.275209,0.970238,0.0157225,0.652778,0.83631
4,ABBV,134.090,149,0.327845,0.611111,0.173697,0.748016,0.241681,0.954365,0.133446,0.984127,0.824405
5,ANTM,453.350,44,0.465755,0.771825,0.194193,0.78373,0.206477,0.926587,0.0448434,0.779762,0.815476
6,AAP,242.960,82,0.521494,0.819444,0.233837,0.861111,0.161659,0.869048,0.0191779,0.674603,0.806052
7,AVB,251.658,79,0.596559,0.882937,0.193254,0.779762,0.111459,0.757937,0.017225,0.660714,0.770337
8,AMP,308.660,64,0.638442,0.914683,0.199289,0.787698,0.111474,0.759921,-0.0280286,0.40873,0.717758
9,ARE,223.100,89,0.256267,0.503968,0.210492,0.829365,0.103856,0.72619,0.0515945,0.809524,0.717262


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [34]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [35]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [36]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [37]:
writer.save()